In [11]:
import sqlite3
import csv
SQLLITEFILE = '/home/jay/BigData/SFUCourses/CMPT733/Project/Data/latlongadd_db.sqlite'

sqlite_file = SQLLITEFILE    # name of the sqlite database file

# Connecting to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# Creating a new SQLite table with 1 column
c.execute("""CREATE TABLE IF NOT EXISTS AddressLatLong (
  IsUpdated integer,
  House text,
  Street text,
  City text,
  Province text,
  Country text,
  PostalCode text,
  Latitude real,
  Longitude real);
""")
c.execute("""CREATE INDEX IF NOT EXISTS queryindex on AddressLatLong (House, Street,City, Province, Country, PostalCode);""")

# Committing changes and closing the connection to the database file
conn.commit()
conn.close()



In [12]:
UNKNOWN_WITH_ADDRESS = '/home/jay/BigData/SFUCourses/CMPT733/Project/Data/unknown_withaddress.csv/part-00000-0b8310d9-f124-4261-b7c3-eef6188b1694.csv'
import sqlite3
import csv

SQLLITEFILE = '/home/jay/BigData/SFUCourses/CMPT733/Project/Data/latlongadd_db.sqlite'

sqlite_file = SQLLITEFILE    # name of the sqlite database file

# Connecting to the database file
conn = sqlite3.connect(sqlite_file)

def format_string(val):
    if val == "":
        val = ""
    else:
        val = val.replace("'","''")
    return val

def format_insert(csvline):
    
    if len(csvline) > 0:
        sql_insert_template = "INSERT INTO AddressLatLong"\
        "(IsUpdated, House, Street, City, Province, Country, PostalCode) " \
        "VALUES(0,'{0}','{1}','{2}','{3}','{4}','{5}');\n"

        if len(csvline) > 0:
            house = format_string(csvline[0])
            street = format_string(csvline[1])
            city = format_string(csvline[2])
            province = format_string(csvline[3])
            country = format_string(csvline[4])
            postalcode = format_string(csvline[5])

            return sql_insert_template.format(house, street, city, province, country, postalcode)
        return ""


def execute_sql_batches(filename):
    c = conn.cursor()
    with open(filename,'r') as file_pointer:
        csvfile = csv.reader(file_pointer)
        csvfile.next()
        for line in csvfile:
                sql_text = format_insert(line)
                c.execute(sql_text)

    #print("All rows areUNKNOWN_WITH_ADDRESS,inserted."+" for file:" + filename + ".....")
    

execute_sql_batches(UNKNOWN_WITH_ADDRESS)
conn.commit()
conn.close()

In [45]:
import sqlite3
import csv
import urllib
import json

SQLLITEFILE = '/home/jay/BigData/SFUCourses/CMPT733/Project/Data/latlongadd_db.sqlite'

sqlite_file = SQLLITEFILE    # name of the sqlite database file

# Connecting to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()
#libraries for OAuth
import requests
from requests_oauthlib import OAuth1

def latlong_from_address(csvline):
     
    address = ' '.join(csvline)
    #keys
    client_key = 'dj0yJmk9ZUtvYVZYaURqNVNGJmQ9WVdrOVQzcEpVRlZ3TXpnbWNHbzlNQS0tJnM9Y29uc3VtZXJzZWNyZXQmeD03NQ'
    client_secret = '59bfb1cbf4272ad5656a10b5660697781ba202b9'

    #Using OAuth1 to make a simple query request on when your application doesn't require any permissions
    baseurl = "https://query.yahooapis.com/v1/public/yql?"
    yql_query = "select centroid from geo.places where text=\""+address+"\""
    yql_url = baseurl + urllib.urlencode({'q':yql_query}) + "&format=json"
    queryoauth = OAuth1(client_key, client_secret, signature_type='query')
    r = requests.get(url=yql_url, auth=queryoauth)

    result = json.loads(r.content.decode('utf8'))
    latitude =0
    longitude = 0

    if result['query']['count'] > 1:
        latitude = result['query']['results']['place'][0]['centroid']['latitude']
        longitude = result['query']['results']['place'][0]['centroid']['longitude']
    elif result['query']['count'] == 1:
        latitude = result['query']['results']['place']['centroid']['latitude']
        longitude = result['query']['results']['place']['centroid']['longitude']
    elif result['query']['count'] == 0:
        print "nO resut"
    
    return latitude, longitude

def format_string(val):
    if val == "":
        val = ""
    else:
        val = val.replace("'","''")
    return val

def format_update(csvline, latitude, longitude):
    
    if len(csvline) > 0:
        sql_insert_template = """UPDATE AddressLatLong 
        SET Latitude={0}, Longitude={1}, IsUpdated=1 
        WHERE House='{2}' AND Street ='{3}'
        AND City='{4}' AND Province='{5}' AND Country = '{6}'
        AND PostalCode ='{7}'
        """

        if len(csvline) > 0:
            house = format_string(csvline[0])
            street = format_string(csvline[1])
            city = format_string(csvline[2])
            province = format_string(csvline[3])
            country = format_string(csvline[4])
            postalcode = format_string(csvline[5])

            return sql_insert_template.format(latitude, longitude,
                                              house, street, city, province, country, postalcode                                              )
        return ""



def update_record(csvline):
    lat, lon = latlong_from_address(csvline)
    sql_text = format_update(csvline, lat, lon)
    c.execute(sql_text)


total_count = 2000

    
def update_until_finish():   
    counter = 0
    csvline =['1']
    while counter < total_count:
        counter += 1
        c.execute("SELECT House, Street, City, Province, Country, PostalCode FROM AddressLatLong WHERE IsUpdated=0 LIMIT 1")
        csvline = c.fetchall()
        if len(csvline) > 0:
            csvline = list(csvline[0])
        else:
            break
        update_record(csvline)
        conn.commit()
    else:
        return "Done"

print update_until_finish()
conn.commit()
conn.close()

IndexError: list index out of range